### Наивный байесовский классфикатор.

Многие из вас могли слышать про [теорему Байеса](http://mathprofi.ru/formula_polnoj_verojatnosti_formuly_bajesa.html) из курса матстата и теории вероятностей:

$\large P(A_i|B) = \frac{P(A_iB)}{P(B)} =  \frac{P(B \mid A_i)\, P(A_i)}{P(B)}$

Кратко опишем смысл формулы:  

_Пусть некое событие $B$ может наступить в результате осуществеления одной из гипотез $A_1$, $A_2$, $A_3$ и тд.  
Зная вероятности гипотез $A$ до наступления события, можно, уже после свершения события $B$, вычислить, какая из гипотез привела к свершению этого события с наибольшей вероятностью._



В примере выше у нас был только один признак (курение), но что делать, если признаков больше одного?  

Формула изменится таким образом:

$ P(A_i|B_1, B_2, ... , B_n) = \frac{P(B_1, B_2, ... , B_n | A_i)*P(A_i)}{P(B)}$

Условная вероятность в числителе расписывается на произведение вероятностей. Тут есть важный нюанс - мы наивно предполагаем, что признаки $B_1$, $B_2$, ... , $B_n$ независимы друг от друга, то есть они никак не коррелируют друг с другом:

$ P(B_1, B_2, ... , B_n | A_i) = P(B_1 | A_i) * P(B_2 | A_i) *\ ...\ * P(B_n | A_i) $

Значит, перед использованием классификатора в идеале необходимо проверить все признаки на взаимную корреляцию, и удалить сильно коррелирующие. Для поиска можно использовать [критерий Пирсона](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)

Априорные и апостериорные вероятности и так далее - это конечно интересно, но где тут связь с классификацией?! Вспомним типичную задачку на теорему байеса:  

**_В выборке 40% мужчин и 60% женщин. Известно, что среди них курит 10% женщин и 70% мужчин. Про некоего человека N известно, что он курит. Мужчина он, или женщина?_**

Данная задачка по сути есть классификация - у нас есть значение нецелевого признака (курение), и на основе этого нам надо предсказать пол. Решение задачки очень простое, так как все нужные нам числа прописаны сразу в условии:

$P(M) = 0.4$  
$P(Ж) = 0.6$  
$P(K|M) = 0.7$ - вероятность того, что человек курит (К) если он мужчина.  
$P(К|Ж) = 0.1$ - вероятность того, что человек курит, если он женщина.  

Понятно, что является человек мужчиной или женщиной - это гипотезы, а вероятность того, что курит или не курит - событие, которое может произойти в результате наступления этих гипотез.  
Рассчитаем полную вероятность того, что человек курит:
$P(К) = P(M)*P(K|M) + P(Ж)*P(К|Ж) = 0.34$.

Теперь мы сможем наконец предсказать пол человека зная, курит он (она?) или нет:

$P(M|К) = \frac{P(К|М)*P(M)}{P(К)} = 28/34$

$P(Ж|К) = \frac{P(К|Ж)*P(Ж)}{P(К)} = 6/34$

Видно, что тут у нас есть вероятности всех классов. Выбираем просто класс с наибольшей вероятностью (argmax) - и это мужчина. Очевидный плюс такого классифкатора - не требуется большой объем данных.

Любопытно, что если реализовывать классификатор "в лоб", по определению теоремы байеса, то точность его будет не высока, так как не учитывается то, каким образом распределены сами данные. Подробно про разные варианты байесовского классификатора можно почитать [тут](https://scikit-learn.org/stable/modules/naive_bayes.html). В данной работе мы будем реализовывать [несколько упрощенный мультиномиальный классификатор](http://bazhenov.me/blog/2012/06/11/naive-bayes.html). Настоятельно рекомендую **прочесть пример** по ссылке и разобраться, каким образом высчитываются все параметры. 

### Пред-подготовка данных

Для начала мы разберемся с данными, а затем реализуем классификатор.

Говоря о самих данных, то мы попробуем решить задачу классификации [новостных текстов](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) - байесовские классификаторы часто используются в этой прикладной области машинного обучения.

In [2]:
# посмотрим на данные - в этот раз датасет доступен напрямую из sklearn.
# выведем все доступные категории.
# параметр subset отвечает за разделенение данных (train - тренировочная выборка, test - тестовая).

# параметр remove говорит о том, какие части данных нужно удалить, чтобы не допустить переобучения.
# headers - заголовки новостных групп
# quotes - удаление строк, похожих на цитаты из других источников
# footers - удаление блоков из конца текста, похожих на подписи

from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Мы будем использовать только 4 класса текстов: `alt.atheism`, `sci.space`, `talk.religion.misc`, `comp.graphics`.  Используя параметр `categories` в функции `fetch_20newsgroups`, задайте список нужных нам категорий и разбейте данные на тренировочную и тестовые части (параметр `subset`). 

Учтите, что сами данные (целевые и нецелевые признаки) лежат в атрибутах `data` и `target`:
```python
subset = fetch_20newsgroups( ... )
X = subset.data
y = subset.target
```

In [3]:
categories = ('alt.atheism', 'sci.space', 'talk.religion.misc', 'comp.graphics')


newsgroups_train = fetch_20newsgroups(
    subset='train',
    remove=('headers', 'footers', 'quotes'),
    categories = categories)

newsgroups_test = fetch_20newsgroups(
    subset='test',
    remove=('headers', 'footers', 'quotes'),
    categories = categories)

x_train = newsgroups_train.data
x_test = newsgroups_test.data
y_train = newsgroups_train.target
y_test = newsgroups_test.target

Посмотрим на типы данных: видно, что X - это список со строками, а y - просто массив с метками класса.

In [4]:
print(type(x_train))
print(type(x_train[0]))
print(type(y_train))
print(type(y_train[0]))

<class 'list'>
<class 'str'>
<class 'numpy.ndarray'>
<class 'numpy.int64'>


Выведите на экран по 1 тексту из каждой категории.

In [5]:
for category_idx, category_name in enumerate(newsgroups_train.target_names):
    print(f"Категория: {category_name}")
    print("=" * 50)

    # Находим индекс первого текста в текущей категории
    first_text_idx = next(i for i, label in enumerate(newsgroups_train.target) if label == category_idx)

    # Выводим текст
    print(newsgroups_train.data[first_text_idx])
    print("\n" + "=" * 50 + "\n")

Категория: alt.atheism
I have a request for those who would like to see Charley Wingate
respond to the "Charley Challenges" (and judging from my e-mail, there
appear to be quite a few of you.)  

It is clear that Mr. Wingate intends to continue to post tangential or
unrelated articles while ingoring the Challenges themselves.  Between
the last two re-postings of the Challenges, I noted perhaps a dozen or
more posts by Mr. Wingate, none of which answered a single Challenge.  

It seems unmistakable to me that Mr. Wingate hopes that the questions
will just go away, and he is doing his level best to change the
subject.  Given that this seems a rather common net.theist tactic, I
would like to suggest that we impress upon him our desire for answers,
in the following manner:

1. Ignore any future articles by Mr. Wingate that do not address the
Challenges, until he answers them or explictly announces that he
refuses to do so.

--or--

2. If you must respond to one of his articles, include wit

проведем небольшой эксперимент по отбору признаков: датасет изкоробочный, специально для обучения машинному обучению (sic), НО...  
... проверим данные на наличие пробелов и пустых строк

In [6]:
print(x_train.count(''))
print(x_train.count(' '))
print(x_train.count('  '))

47
4
0


Как мы видим, среди признаков внезапно оказались пустые строки, и этим пустым строкам присвоен класс! Очевидно, что такое недопустимо, поэтому датасет необходимо немного вычистить.  

Заметьте, что мы делаем проверку на строку с 1м, 2мя, 3мя пробелами, но не с 5 пробелами и больше. Чтобы эффективно найти строки с некоторым неизвестным числом пробелов, чтоит воспользоваться регулярным выражением `^\\s*$` - данная регулярка срабатывает на строках, состоящих целиком из пробелов (`\s` - это пробельный символ, квантификатор `*` указывает, что число повторений такого символа больше одного, символ `^` указывает на начало строки, а знак доллара- на конец строки).

Для нахождения возьмем функцию `match` из библиотеки `re` регулярных выражений в питоне.  
Доки по функции [match](https://docs.python.org/3/library/re.html#re.match). Обратите внимание, что она возвратит `None`, если паттерн не совпал с заданной строкой, или возвратит некий `match object`, если будет совпадение.  

**Задача:** в тестовой и тренировочной выборках найти индексы пробельных строк. Зная индексы (это должен быть массив индексов), можно удалить такие элементы из тренировочной и тестовой выборок. Для удаления можете использовать логические маски, или [np.delete](https://numpy.org/doc/stable/reference/generated/numpy.delete.html)

**Hints**
- np.delete
- re.match
- '''^\\s*$'''

Напечатаем число элементов до очистки

In [7]:
print(len(y_train))
print(len(y_test))

2034
1353


In [8]:
import re
import numpy as np 

pattern = re.compile('^\s*$')

# Функция для поиска индексов пробельных строк
def find_empty_indices(data):
    empty_indices = [i for i, text in enumerate(data) if pattern.match(text)]
    return empty_indices

# Находим индексы пробельных строк в тренировочной и тестовой выборках
empty_indices_train = find_empty_indices(x_train)
empty_indices_test = find_empty_indices(x_test)

# Удаляем пробельные строки из тренировочной и тестовой выборок
x_train_cleaned = np.delete(x_train, empty_indices_train)
y_train_cleaned = np.delete(y_train, empty_indices_train)

x_test_cleaned = np.delete(x_test, empty_indices_test)
y_test_cleaned = np.delete(y_test, empty_indices_test)

# Проверка результатов
print(f"Удалено {len(empty_indices_train)} пробельных строк из тренировочной выборки.")
print(f"Удалено {len(empty_indices_test)} пробельных строк из тестовой выборки.")

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
C:\Temp\ipykernel_13772\3227690409.py:4: SyntaxWarning: invalid escape sequence '\s'
  pattern = re.compile('^\s*$')


Удалено 57 пробельных строк из тренировочной выборки.
Удалено 35 пробельных строк из тестовой выборки.


Выведем число элементов после очистки, должно получиться 1977 и 1318 элементов в тренировочной и тестовой выборках:

In [9]:
# Проверка размеров данных после очистки
print(f"Размер x_train после очистки: {len(x_train_cleaned)}")
print(f"Размер y_train после очистки: {len(y_train_cleaned)}")
print(f"Размер x_test после очистки: {len(x_test_cleaned)}")
print(f"Размер y_test после очистки: {len(y_test_cleaned)}")

# Обновление утверждений
assert len(y_train_cleaned) == len(x_train_cleaned), "Размеры x_train и y_train не совпадают"
assert len(y_test_cleaned) == len(x_test_cleaned), "Размеры x_test и y_test не совпадают"

Размер x_train после очистки: 1977
Размер y_train после очистки: 1977
Размер x_test после очистки: 1318
Размер y_test после очистки: 1318


Посмотрим на то, что из себя представляют целевые признаки. Это целое число, обозначающее индекс категории.

In [10]:
np.unique(y_train)

array([0, 1, 2, 3], dtype=int64)

Преобразуем этот индекс в имя категории. Для этого воспользуемся генератором списков и методом `target_names`, который по индексу вернет нам название категории.

In [11]:
y_test = [newsgroups_test.target_names[idx] for idx in y_test]
y_train = [newsgroups_train.target_names[idx] for idx in y_train]

In [12]:
np.unique(y_train)

array(['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc'],
      dtype='<U18')

### Как извлечь информацию из текста

В предыдущей части вы подготовили сам датасет, состоящий из строк с символами. Но каким образом конвертировать символы в числа, чтобы предложение образовало точку в некотором многомерном пространстве?

Для этого есть несколько способов:
- Счетчики
- Распределенная семантика

**Счетчики** работают довольно просто - из слов всех предложений выборки вы формируете один большой словарь, и, получая новое предложение, просто считаете, сколько раз каждое слово из словаря было представлено в этом предложении. Таким образом, ваше предложение закодировано вектором, длина которого равна длине словаря. Недостатков у этого метода только два: получающийся вектор имеет очень большую длину, и к тому же он сильно разряжен (в нем много нулей, так как одно предложение априори не может содержать всех слов), из-за чего с ним трудно работать. Второй недостаток - мы просто смотрим на сам _факт наличия_ слова в предложении, но не на _контекст_ этого слова.

Оба этих недостатка успешно забарывают методы, основанные на **распределенной семантике**, такие как легендарный Word2Vec и известный GloVe. Такие методы основаны на недо-нейронных сетях, поволяют учитывать контекст слова, и позволяют создавать вектора заданной пользователем длины. 

Проблема их в том, что значения таких векторов - это и отрицательные числа, а байесовская модель работает только с натуральными. К тому же, такие алгоритмы работают на уровне слова, а не предложения - у вас просто будет набор векторов, представляющих каждое слово из предложения, а каким образом представить вектор самого предложения с наименьшими потерями информации - вопрос.  Поэтому далее мы будем использовать именно счетчики.

Для работы со счетчиками мы возьмем реализацию из библиотеки `sklearn`.  

по ссылке [тут](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) можно почитать про сами счетчики (мешок слов и TF-IDF)

#### **Мешок слов**

[Документация](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)

Можно начать с очень простой идеи. Давайте разобъем все предложения на слова. Составим словарь всех слов, которые будут встречаться во всех  наших текстах. И отметим, встречается ли это слово в нашем конкретном примере. Другими словами, пусть в таблице в строках будут предложения, в столбцах - слова, а в ячейках число, которое показывает сколько раз это слово встречалось в этом предложении. Получается, что каждому объекту выборки будет сопоставлен вектор.

Векторизацию мы делаем сразу методом `fit_transform` - он эквивалентен последовательному вызову 
```python
bow = count_vectorizer.fit(data).transform(data)
```

Очевидно, что метод `fit` составляет словарь, а `transform` делает вектор из предложения, согласно имеющемуся словарю.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
texts = [
    "I've been searching for the right words to thank you for this breather.",
    "You have been wonderful and a blessing at all times",
    "I promise i wont take your help for granted and will fulfil my promise."
]
bow = count_vectorizer.fit_transform(texts)
print("Shape=", bow.shape)

Shape= (3, 28)


In [14]:
# посмотрим на словарь всех слов (метод vocabulary_)
# число - это индекс слова в строке матрицы

count_vectorizer.vocabulary_

{'ve': 21,
 'been': 3,
 'searching': 14,
 'for': 6,
 'the': 17,
 'right': 13,
 'words': 25,
 'to': 20,
 'thank': 16,
 'you': 26,
 'this': 18,
 'breather': 5,
 'have': 9,
 'wonderful': 23,
 'and': 1,
 'blessing': 4,
 'at': 2,
 'all': 0,
 'times': 19,
 'promise': 12,
 'wont': 24,
 'take': 15,
 'your': 27,
 'help': 10,
 'granted': 8,
 'will': 22,
 'fulfil': 7,
 'my': 11}

Теперь составим ту самую матрицу, где в столбцах слова, а в строках тексты.

Как мы видим, в первом и втором предложениях есть слово "been", а в третьем его нет (так как у "been" индекс равен 3).  
Так как векторайзер возвращает разряженную матрицу, то воспользуемся методом `.toarray()`, чтобы превратить ее в numpy-массив.

In [15]:
bow.toarray()

array([[0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
        0, 0, 0, 1, 1, 0],
       [1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 1]], dtype=int64)

При векторизации можно удалить "стоп-слова" - они не несут какого-то смысла, но нужны для грамматики (параметр `stop_words`). Как мы видим, словарь стал заметно меньше, соответсвенно и вектор тоже стал короче.

In [16]:
count_vectorizer = CountVectorizer(stop_words='english')
bow = count_vectorizer.fit_transform(texts)
print("Shape=", bow.shape)
count_vectorizer.vocabulary_

Shape= (3, 14)


{'ve': 10,
 'searching': 7,
 'right': 6,
 'words': 13,
 'thank': 8,
 'breather': 1,
 'wonderful': 11,
 'blessing': 0,
 'times': 9,
 'promise': 5,
 'wont': 12,
 'help': 4,
 'granted': 3,
 'fulfil': 2}

#### **TF-IDF**

[Документация в sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

Мешок слов не учитывает "веса" слов, он просто смотрит их вхождение в документ. Вероятно, было бы полезно взвесить каким-то образом каждое слово в документе. Действительно, если слово встречается во всех документах, то, наверное, его вес небольшой. А если редкое слово встречается в некоторых документах, то скорее всего оно какое-то узко тематическое.

Один из способов взвесить слова - это использовать меру tf-idf, где:

**TF - term frequency** - частота слова для каждой статьи

$$\LARGE \mathrm{tf}(t,d) = \frac{n_t}{\sum_k n_k}$$

**IDF - inverse document frequency*** — обратная частота документа - уменьшает вес часто встречаемых слов

$$\LARGE \mathrm{idf}(t, D) =  \log \frac{|D|}{|\{\,d_i \in D \mid t \in d_{i}\, \}|}$$

$|D|$ - число документов в корпусе

$|\{\,d_i \in D \mid t \in d_{i}\, \}|$ - число документов из коллекции ${\displaystyle D}$ , в которых встречается ${\displaystyle t}$  (когда ${\displaystyle n_{t}\neq 0}$ ).

**TF-IDF**

$$\LARGE \operatorname{tf-idf}(t,d,D) = \operatorname{tf}(t,d) \times \operatorname{idf}(t, D)$$


Синтаксис такой же, как и у мешка слов

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
texts = [
    "I've been searching for the right words to thank you for this breather.",
    "You have been wonderful and a blessing at all times",
    "I promise i wont take your help for granted and will fulfil my promise."
]
bow = tfidf_vectorizer.fit_transform(texts)
print("Shape=", bow.shape)

Shape= (3, 14)


In [18]:
tfidf_vectorizer.vocabulary_

{'ve': 10,
 'searching': 7,
 'right': 6,
 'words': 13,
 'thank': 8,
 'breather': 1,
 'wonderful': 11,
 'blessing': 0,
 'times': 9,
 'promise': 5,
 'wont': 12,
 'help': 4,
 'granted': 3,
 'fulfil': 2}

In [19]:
bow.toarray()

array([[0.        , 0.40824829, 0.        , 0.        , 0.        ,
        0.        , 0.40824829, 0.40824829, 0.40824829, 0.        ,
        0.40824829, 0.        , 0.        , 0.40824829],
       [0.57735027, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.57735027,
        0.        , 0.57735027, 0.        , 0.        ],
       [0.        , 0.        , 0.35355339, 0.35355339, 0.35355339,
        0.70710678, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.35355339, 0.        ]])

#### **Задание**

Обратясь к примерам выше и к документации по ссылкам, создайте векторайзеры для подготовленных в предыдущем параграфе данных. Используйте английские стоп-слова. 

Так как у нас есть две части (тренировочная и тестовая выборки), то векторайзер нужно обучить на словах из обоих выборок (подумайте, почему). Так как у нас питон, а наши выборки это массивы строк, то объеденить их очень просто - просто сложить. После того, как вы обучили векторайзер на всех словах, проведите трансформации отдельно для тестовой, и отдельно для тренировочных частей.

Векторизованные части **назовите** `xcv_test/train` для count_vectorizer, и `xTfidf_test/train` для TF-IDF - это нужно для корректной работы тестов и примеров ниже.

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Объединяем тренировочную и тестовую выборки
all_texts = np.concatenate((x_train_cleaned, x_test_cleaned))

# Создаём и обучаем CountVectorizer
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer.fit(all_texts)

# Преобразуем тренировочную и тестовую выборки
xcv_train = count_vectorizer.transform(x_train_cleaned)
xcv_test = count_vectorizer.transform(x_test_cleaned)

# Создаём и обучаем TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_vectorizer.fit(all_texts)

# Преобразуем тренировочную и тестовую выборки
xTfidf_train = tfidf_vectorizer.transform(x_train_cleaned)
xTfidf_test = tfidf_vectorizer.transform(x_test_cleaned)

# Проверка размеров
print("CountVectorizer:")
print(f"xcv_train shape: {xcv_train.shape}")
print(f"xcv_test shape: {xcv_test.shape}")

print("\nTfidfVectorizer:")
print(f"xTfidf_train shape: {xTfidf_train.shape}")
print(f"xTfidf_test shape: {xTfidf_test.shape}")

CountVectorizer:
xcv_train shape: (1977, 33529)
xcv_test shape: (1318, 33529)

TfidfVectorizer:
xTfidf_train shape: (1977, 33529)
xTfidf_test shape: (1318, 33529)


Небольшой тест на проверку размерностей

In [21]:
# count vectorizer
assert xcv_train.shape == (1977, 33529)
assert xcv_test.shape == (1318, 33529)

#tf-idf
assert xTfidf_train.shape == (1977, 33529)
assert xTfidf_test.shape == (1318, 33529)

<a name="clf-description"></a>
### Реализация классификатора

Вспомните статью из блога, которая была в самом начале.

Модель классификатора строится на основе обучающей выборки. 
По ней необходимо найти следующюю статистику:  
1. Частоты классов в корпусе объектов (сколько объектов принадлежит каждому из классов)  (`classes_stats`)
2. Cуммарное число слов в документах каждого класса (`words_per_class`, далее см. $L_c$)
3. Частоты слов в пределах каждого класса (`word_freqs_per_class`, далее используется для расчета $W_{ic}$)
4. Размер словаря выборки (число признаков) - кол-во уникальных слов в выборке (`num_features`)

По сути, это метод `fit` классификатора.

На этапе предсказания необходимо воспользоваться следующей формулой:

$$
\begin{equation}
predicted\ class = \operatorname*{arg\max}_{c \in C} \left[\log{{D_c} \over {D}} + \sum_{i \in Q}{\log{{W_{ic} + 1} \over {|V| + L_c}}}\right]
\end{equation}
$$

Поясним некоторые переменные в этом выражении:  
$D_c$ - количество документов в обуч. выборке $\in$ классу $c$

$D$ - сколько всего было документов в обуч. выборке

$|V|$ - количество уникальных слов во сех документах обуч. выборки

$L_c$ - cуммарное число слов в документах класса $c$ обучающей выборки
 
$W_{ic}$ - сколько раз $i$ слово встретилось в объектах класса $c$ обучающей выборки

$Q$ - множество слов _классифицируемого_ документа

Сигнатура класса:

```python
class NaiveBayes:
    def fit(self, x, y) -> None
    
    def predict(self, x) -> List[Int]
```

Для начала отдельно подсчитаем различные статистики, описанные в статье и в материале выше. 
Так как у нас уже готовы все данные, то считать будем по **count_vectorizer**'у (то есть xcv_ ...).

Общее число документов в обучающей выборке (`doc_num`)

In [22]:
doc_num = x_train_cleaned.shape[0]
print(doc_num)

1977


In [23]:
# ПРОВЕРКА
assert doc_num == 1977

Словарь, содержащий число объектов каждого класса (`classes_stats`)

In [24]:
from collections import Counter

# Создаем словарь с количеством объектов каждого класса в тренировочной выборке
class_indices = y_train_cleaned
class_names = newsgroups_train.target_names

# Считаем количество объектов каждого класса
class_counts = Counter(class_indices)

# Создаем словарь, где ключи — имена классов, а значения — количество объектов
classes_stats = {class_names[idx]: count for idx, count in class_counts.items()}

# Выводим результат
print("Статистика по классам в тренировочной выборке:", classes_stats)

Статистика по классам в тренировочной выборке: {'comp.graphics': 571, 'talk.religion.misc': 361, 'sci.space': 577, 'alt.atheism': 468}


In [25]:
# ПРОВЕРКА
assert classes_stats['alt.atheism'] == 468
assert classes_stats['comp.graphics'] == 571
assert classes_stats['sci.space'] == 577
assert classes_stats['talk.religion.misc'] == 361

Число уникальных признаков (слов) в тренировочной выборке (`num_features`)

In [26]:
num_features = xcv_train.shape[1]

# Вывод результата
print("Число уникальных признаков (слов) в тренировочной выборке:", num_features)

Число уникальных признаков (слов) в тренировочной выборке: 33529


In [27]:
# ПРОВЕРКА
assert num_features == 33529

Создадим словарь `indexes`, в котором ключом будет являться имя класса, а значением - список строк матрицы X, принадлежащих этому классу. Этот список пригодится нам дальше, так как будет играть роль маски. Для поиска класса каждой из строк используйте целевой вектор.

**Hints:**
- [np.where](https://numpy.org/doc/stable/reference/generated/numpy.where.html) 

In [28]:
import numpy as np

# Создаём словарь indexes
indexes = {}

# Проходим по всем уникальным классам
for category_idx, category_name in enumerate(newsgroups_train.target_names):
    # Находим индексы строк, принадлежащих текущему классу
    class_indices = np.where(y_train_cleaned == category_idx)[0]
    
    # Добавляем в словарь
    indexes[category_name] = class_indices

# Пример использования: выбор строк для класса 'alt.atheism'
atheism_indices = indexes['alt.atheism']
print(f"Количество строк для класса 'alt.atheism': {len(atheism_indices)}")

Количество строк для класса 'alt.atheism': 468


In [29]:
# ПРОВЕРКА
# так как в словаре очень много элементов, то проверим случайные элементы из списка.
# если вы все сделали правильно, то эти элементы совпадут.
assert indexes['sci.space'][35] == 111
assert indexes['comp.graphics'][42] == 159
assert indexes['talk.religion.misc'][67] == 312
assert indexes['alt.atheism'][89] == 372

Используя найденные выше индексы, подсчитаем два важных параметра `words_per_class` и `word_freqs_per_class`.  
Обе этих переменных являются словарями, но первая из них отвечает за суммарное число слов, использованных в **каждом классе**, а вторая показывает, сколько раз конкретное слово встретилось в документах определенного класса.  Соответственно, формат переменной `words_per_class` - `{str: int}`, формат `word_freqs_per_class` - `{str: List}`.
Мы специально объеденили поиск двух разных статистик в одном блоке, чтобы избежать лишних циклов.

Чтобы найти в X строки, относящиеся к тому или иному классу, воспользуйтесь поиском по маске `indexes` для нужного класса.

Также помните, что X - это разряженная матрица, но из нее можно получить обычный список через метод `toarray()`

**Hints::**
- вспомните про маски в numpy-массивах
```python
mask = [1,0,2] #indexes
array = [1,2,4,8,16,32,64,128]
array[mask]
#result: array([2, 1, 4])
```

In [30]:
x_arr = xcv_train.toarray()

words_per_class = {}
word_freqs_per_class = {}

for cls in indexes.keys():
    class_idxs = indexes[cls] # нашли индексы строк матрицы, относящихся к классу cls

    subarray_rows = x_arr[class_idxs] # нашли подмассив, относящийся к  классу cls
    subarray_sum = np.sum(subarray_rows, axis = 0) # провели суммирование по столбцам
    word_freqs_per_class[cls] = subarray_sum

    words_per_class[cls] = len(subarray_sum[subarray_sum != 0]) # узнали,
        # сколько слов было использовано в рамках одного класса, 
        # то есть просто подсчитали число ненулевых элементов

words_per_class, word_freqs_per_class

({'alt.atheism': 8737,
  'comp.graphics': 10592,
  'sci.space': 13273,
  'talk.religion.misc': 8860},
 {'alt.atheism': array([ 0, 17,  0, ...,  0,  0,  0], dtype=int64),
  'comp.graphics': array([26, 12,  0, ...,  0,  0,  2], dtype=int64),
  'sci.space': array([32, 92,  2, ...,  0,  0,  0], dtype=int64),
  'talk.religion.misc': array([1, 9, 0, ..., 0, 0, 0], dtype=int64)})

Все вышенаписанные переменные образуют метод `fit` будущего классификатора. Теперь внесите этот код в метод fit, и не забудьте сделать найденные переменные полями экземпляра класса при помощи `self`

In [31]:
class NaiveBayes:
    def __init__(self):
        # Инициализация полей
        self.indexes = {}  # Словарь для хранения индексов строк по классам
        self.words_per_class = {}  # Словарь для хранения количества уникальных слов по классам
        self.word_freqs_per_class = {}  # Словарь для хранения частот слов по классам
        self.vectorizer = None  # Векторизатор для преобразования текста в числовой формат
        self.class_probs = {}  # Априорные вероятности классов

    def fit(self, x_train, y_train):
        """
        Метод для обучения классификатора.
        
        :param x_train: Текстовые данные для обучения (список строк).
        :param y_train: Метки классов для обучения.
        """
        # Создаём и обучаем CountVectorizer
        self.vectorizer = CountVectorizer(stop_words='english')
        x_arr = self.vectorizer.fit_transform(x_train).toarray()  # Преобразуем тексты в числовой формат

        # Вычисляем априорные вероятности классов
        total_samples = len(y_train)
        unique_classes, class_counts = np.unique(y_train, return_counts=True)
        for cls, count in zip(unique_classes, class_counts):
            self.class_probs[cls] = count / total_samples

        # Создаём словарь indexes
        for cls in unique_classes:
            class_idxs = np.where(y_train == cls)[0]  # Индексы строк, относящихся к классу cls
            self.indexes[cls] = class_idxs

            # Подсчёт частот слов и количества уникальных слов для каждого класса
            subarray_rows = x_arr[class_idxs]  # Подмассив строк, относящихся к классу cls
            subarray_sum = np.sum(subarray_rows, axis=0)  # Суммирование по столбцам
            self.word_freqs_per_class[cls] = subarray_sum  # Частоты слов
            self.words_per_class[cls] = len(subarray_sum[subarray_sum != 0])  # Количество уникальных слов

    def predict(self, x_test):
        """
        Метод для предсказания классов на новых данных.
        
        :param x_test: Текстовые данные для предсказания (список строк).
        :return: Предсказанные метки классов.
        """
        # Преобразуем текстовые данные в числовой формат
        x_test_arr = self.vectorizer.transform(x_test).toarray()

        # Список для хранения предсказаний
        predictions = []

        # Проходим по каждому документу в тестовой выборке
        for doc in x_test_arr:
            class_scores = {}

            # Вычисляем вероятность для каждого класса
            for cls in self.indexes.keys():
                # Логарифм априорной вероятности класса
                log_prob = np.log(self.class_probs[cls])

                # Логарифм правдоподобия (с учётом частот слов)
                word_freqs = self.word_freqs_per_class[cls]
                total_words_in_class = np.sum(word_freqs)
                vocab_size = len(self.vectorizer.vocabulary_)

                # Вычисляем логарифм правдоподобия с использованием сглаживания Лапласа
                likelihood = np.sum(doc * np.log((word_freqs + 1) / (total_words_in_class + vocab_size)))

                # Общий счёт для класса
                class_scores[cls] = log_prob + likelihood

            # Выбираем класс с максимальным счётом
            predicted_class = max(class_scores, key=class_scores.get)
            predictions.append(predicted_class)

        return np.array(predictions)

Теперь реализуем метод `predict`. Вспомните еще раз [большую формулу](#clf-description) из начала этого раздела. Если вы внимательно читали статью, то заметили, что в примере мы получили не чистые вероятности классов, а всего лишь числовые оценки. Далее эти оценки можно перевести в вероятности, но мы этого делать не будем.

Тогда промежуточный выход классификатора обозначим так:

`pred_per_class = {<номер строки в тестовой выборке>: {<класс 1>: <оценка>, ... , <класс n>: <оценка>}}`

Таким образом итоговый класс к которому будет отнесена строка - просто класс с наибольшей оценкой

Не забудьте о следующих вещах:
- X_test - это такая же разряженная матрица, нужно превратить ее в список
- переменная $\frac{D_c}{D}$ одинакова для одного класса.
- Чтобы подсчитать $W_{ic}$ воспользуйтесь `word_freqs_per_class`.
- Чтобы понять, какие элементы вам нужно брать в `word_freqs_per_class`, найдите индексы ненулевых элементов в классифицируемой строке - если эти элементы ненулевые, значит, там было какое-то слово


**Hints:**
- [np.nonzero](https://numpy.org/doc/stable/reference/generated/numpy.nonzero.html) (будьте внимательны с типом возвращаемого значения!)
- [Math.log](https://docs.python.org/3/library/math.html) для чисел
- [np.log](https://numpy.org/doc/stable/reference/generated/numpy.log.html) для массивов
- Так как с каждым разом вы добавляете оценку для нового класса, логично использовать [defaultdict(dict)](https://docs.python.org/3/library/collections.html#collections.defaultdict): так создается словарь, состоящий из пустых словарей, и при обращении к элементу словаря, например `pred_per_class['SomeID']` мы получаем словарь, для которого доступны все стандартные методы (например, update)

In [32]:
# Пример использования
classifier = NaiveBayes()
classifier.fit(x_train_cleaned, y_train_cleaned)

# Предсказание на тестовых данных
y_pred = classifier.predict(x_test_cleaned)

# Проверка размерности предсказаний
assert len(y_pred) == len(y_test_cleaned), "Размерность предсказаний не совпадает с размерностью y_test"

### Проверка классификатора

Воспользуемся матрицей ошибок и отчетом классификации из `sklearn`, проверим классификатор и на count_vectorizer-векторах, и на tf-idf-векторах. Для начала поговорим о способах оценки качества классификации.

#### Матрица ошибок

Основной материал можете прочесть в [статье](https://habr.com/ru/company/ods/blog/328372/), тут же напишем кратко.

Рассмотрим матрицу: 
 
 
 <br></br> | $y=1$ | $y=0$
--- | --- | ---
$\overline{y} = 1$ | **True Positive (TP)** | **False Positive (FP)**
$\overline{y} = 0$ | **False Negative (FN)** | **True Negative (TN)**

$y$ - истинная метка класса, $\overline{y}$ - предсказание классификатора.

По главной диагонали матрицы - число правильно классифицированных объектов (TP и FP).  

Понятно, что число классов может быть больше двух, но для простоты будем рассматривать только случай с двумя классами, назовем их условно Positive и Negative.

Ошибки бывают двух типов: **ошибки первого рода** (FP), или ложноположительное срабатывание, когда, например, анализ показывает заболевание, хотя на самом деле человек здоров, и **ошибки второго рода** (FN), или пропуск события, когда больного человека принимают за здорового. 

В некотрых случаях по оси X может отображаться предсказание модели, а по оси Y - истинные метки. Важно не запоминать положение элементов в матрице, а в зависимости от заданных осей понимать, где ошибки, а где правильные значения.

**Метрики**

_Accuracy_, или точность.  
Обозначает в целом долю правильных ответов:

$$
Accuracy = \frac{TP + TN}{TP + FP + TN + FN}
$$

Недостаток метрики в том, что она не работает на несбалансированных выборках.

_Precision_ и _Recall_, или точность и полнота.  

---

**Важное замечание**  
_Оба термина переводятся на русский язык как точность, хотя при этом отражают разные понятия. Чтобы не допустить недопонимая, можно использовать английские слова напрямую, либо accuracy называть аккуратностью, а precision - точностью. Полнота - она и есть полнота._

---

Эти две метрики рассчитываются отдельного для каждого из классов. Для примера рассмотрим класс Positive:

$$
Precision = \frac{TP}{TP + FP}
$$
$$
Recall = \frac{TP}{TP + FN}
$$

Precision отражает долю объектов класса Positive, которые классификатор классифицировал верно.

Recall показывает долю объектов класса Positive среди всех объектов класса Positive, которые вообще нашел алгоритм.

F-мера - один из способов объеденить Precision и Recall:

$$
F_\beta = (1 + \beta^2) \cdot \frac{Precision \cdot Recall}{\beta^2 \cdot Precision + Recall}
$$

$\beta$ - вес Precision в метрике.

В `sklearn` за матрицу ошибок и метрики отвечают [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) и [classification_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html).

In [33]:
from sklearn.metrics import classification_report, confusion_matrix

# Создаём экземпляр классификатора
nb = NaiveBayes()

# Обучаем модель на текстовых данных
nb.fit(x_train_cleaned, y_train_cleaned)  # Передаём текстовые данные, а не xcv_train

# Предсказываем классы для тестовых данных
pred = nb.predict(x_test_cleaned)  # Передаём текстовые данные, а не xcv_test

# Выводим отчёт о классификации
print(classification_report(y_test_cleaned, pred))
print(confusion_matrix(y_test_cleaned, pred))

              precision    recall  f1-score   support

           0       0.65      0.73      0.69       311
           1       0.91      0.92      0.91       384
           2       0.86      0.87      0.86       378
           3       0.66      0.55      0.60       245

    accuracy                           0.79      1318
   macro avg       0.77      0.76      0.77      1318
weighted avg       0.79      0.79      0.79      1318

[[226   7  20  58]
 [ 11 352  18   3]
 [ 24  19 328   7]
 [ 87   8  16 134]]


Проверим классификатор на полученных ранее tf-idf векторах

In [34]:
# Создаём экземпляр классификатора
nb = NaiveBayes()

# Обучаем модель на текстовых данных
nb.fit(x_train_cleaned, y_train_cleaned)  # Используем очищенные данные

# Предсказываем классы для тестовых данных
pred = nb.predict(x_test_cleaned)  # Используем очищенные данные

# Выводим отчёт о классификации
print(classification_report(y_test_cleaned, pred))
print(confusion_matrix(y_test_cleaned, pred))

              precision    recall  f1-score   support

           0       0.65      0.73      0.69       311
           1       0.91      0.92      0.91       384
           2       0.86      0.87      0.86       378
           3       0.66      0.55      0.60       245

    accuracy                           0.79      1318
   macro avg       0.77      0.76      0.77      1318
weighted avg       0.79      0.79      0.79      1318

[[226   7  20  58]
 [ 11 352  18   3]
 [ 24  19 328   7]
 [ 87   8  16 134]]


### Сравним с версией из sklearn

In [35]:
print(f"Размер x_train после очистки: {len(x_train_cleaned)}")
print(f"Размер y_train после очистки: {len(y_train_cleaned)}")
print(f"Размер y_train после очистки: {len(y_train)}")
print(f"Размер x_train после очистки: {len(x_arr)}")

Размер x_train после очистки: 1977
Размер y_train после очистки: 1977
Размер y_train после очистки: 2034
Размер x_train после очистки: 1977


In [38]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=4)
clf.fit(xcv_train, y_train_cleaned)

y_pred = clf.predict(xcv_test)
print(classification_report(y_test_cleaned, y_pred))

print(confusion_matrix(y_test_cleaned, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.75      0.68       311
           1       0.91      0.91      0.91       384
           2       0.83      0.89      0.86       378
           3       0.68      0.42      0.52       245

    accuracy                           0.78      1318
   macro avg       0.76      0.74      0.74      1318
weighted avg       0.78      0.78      0.77      1318

[[232   9  28  42]
 [ 12 351  20   1]
 [ 20  18 335   5]
 [112   9  20 104]]


In [41]:
clf = MultinomialNB(alpha=4)
clf.fit(xTfidf_train, y_train_cleaned)

y_pred = clf.predict(xTfidf_test)
print(classification_report(y_test_cleaned, y_pred))

print(confusion_matrix(y_test_cleaned, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.64      0.62       311
           1       0.84      0.93      0.88       384
           2       0.63      0.92      0.75       378
           3       0.85      0.04      0.09       245

    accuracy                           0.69      1318
   macro avg       0.73      0.63      0.58      1318
weighted avg       0.73      0.69      0.63      1318

[[199  22  88   2]
 [  1 356  27   0]
 [  5  24 349   0]
 [126  20  88  11]]


### Порассуждаем (дополнительно)

Как вы заметили, качество предсказаний неоднородно (какие-то классы определяются хорошо, какие-то не очень хорошо). В чем может быть причина этого? 

Самостоятельно попробуйте найти топ-10 наиболее часто встречающихся слов в каждой категории, посмотрите на эти слова и напишите свои идеи, почему же при классификации точность падает на той или иной категории.

**_Поле для ответа_**

 . . .